# FACT-UVA: Man is to Programmer as Woman is to Homemaker?

## Links

Debiaswe: https://github.com/tolga-b/debiaswe
Lipstick: https://github.com/gonenhila/gender_bias_lipstick

### How to get the GoogleNews word2vec embeddings:
Download it directly from the official [website](https://code.google.com/archive/p/word2vec/) or clone [this github repo](https://github.com/mmihaltz/word2vec-GoogleNews-vectors). Place the downloaded **.bin** file in the embeddings folder.

### How to get the Glove embeddings:
Go to the official [website](https://nlp.stanford.edu/projects/glove/). Download **glove.840B.300d.zip**. Place the downloaded **.txt** file in the embeddings folder.

## Debiasing Word Embeddings

### Word2vec

The code block bellow executes the main debias function using the word2vec Google News embeddings. Additionally, the function takes as arugments several json files with definitional pairs and geneder specific words as described in the original paper. The function outputs two files - **bias_word2vec.bin** and **debiased_word2vec.bin**, which correspond to the embeddings before and after debiasing.

In [10]:
# Debias word2vec embeddings
!cd code && python3 main.py --debias_o_em=../embeddings/debiased_word2vec.bin --bias_o_em=../embeddings/bias_word2vec.bin

Namespace(bias_o_em='../embeddings/bias_word2vec.bin', debias_o_em='../embeddings/debiased_word2vec.bin', def_fn='../data/definitional_pairs.json', em_limit=50000, eq_fn='../data/equalize_pairs.json', g_words_fn='../data/gender_specific_full.json', i_em='../embeddings/GoogleNews-vectors-negative300.bin', o_ext='bin')
*** Reading data from ../embeddings/GoogleNews-vectors-negative300.bin
/home/yovchev/miniconda3/envs/test/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Number of words:  26391
Saving biased vectors to file...
Debiasing...
Saving to file...


Done!



### Glove

The only difference between the two formats (word2vec and glove) is that the first line of word2vec contains the number of words and the vector size, while glove does no contain said line. In order to simply things and reduce the lenght of the code we can convert one of the two to the other format. This way the code has to supoort only one format. The code block below converts the glove embeddings to the word2vec fromat. Said code block needs to be executed only once.

In [11]:
# convert glove to word2vec format
!cd code/scripts && ./gloveToW2V.sh ../../embeddings/glove.840B.300d.txt ../../embeddings/glove.formatted.txt

extracting number of vectors
there are 2196017 lines
extracting vector dimension
cat: write error: Broken pipe
vectors have size 300
creating word2vec format file
done


After transforming the glove embeddings to the word2vec format we can rerun the previous experiment this time using the glove embeddings. The function will generate two files again - **bias_glove.bin** and **debiased_glove.bin** respectfully.

In [12]:
# Debias glove embeddings
!cd code && python3 main.py --i_em=../embeddings/glove.formatted.txt --debias_o_em=../embeddings/debiased_glove.bin --bias_o_em=../embeddings/bias_glove.bin

*** Reading data from ../embeddings/glove.formatted.txt
/home/yovchev/miniconda3/envs/test/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Number of words:  23177
Saving biased vectors to file...
Debiasing...
Saving to file...


Done!



## Benchmark debiased embeddings

After generating the 4 embeddings files (both biased and debiased for word2vec and glove) we can run the benchmark tests on them to determine if the removing of the biased led to any deterioration. The results from the benchmarks would also show if the results have been replicated using the glove embeddings. The code block bellow evaluates each of the 4 embeddings on all of the benchmark test

In [9]:
!cd code/benchmark/scripts/ && ./run_test.sh

05:00:29 INFO:loading projection weights from /mnt/windows_drive_d/Amsterdam University/Year_1/FACT/embeddings/bias_word2vec.bin
05:00:29 INFO:Loading #26391 words with 300 dim
05:00:30 INFO:Transformed 26391 into 26391 words
05:00:30 INFO:Calculating similarity benchmarks
05:00:30 WARNING:Missing 41 words. Will replace them with mean vector
05:00:30 INFO:Spearman correlation of scores on WS353 0.6488884094043214
05:00:30 WARNING:Missing 105 words. Will replace them with mean vector
05:00:30 INFO:Spearman correlation of scores on MTurk 0.513563416944097
05:00:30 WARNING:Missing 25 words. Will replace them with mean vector
05:00:30 INFO:Spearman correlation of scores on WS353S 0.7197011080164891
05:00:30 WARNING:Missing 17 words. Will replace them with mean vector
05:00:31 INFO:Spearman correlation of scores on SimLex999 0.43510114901981567
05:00:31 WARNING:Missing 23 words. Will replace them with mean vector
05:00:31 INFO:Spearman correlation of scores on WS353R 0.5805558093795756
05:0

In [17]:
# show results
import csv
from tabulate import tabulate
from statistics import variance as var

def show_benchmarks(file):
    with open(file, 'r') as f:
        rows = list(csv.reader(f))[:-1]
        rows = [list(x) for x in zip(*rows)]
        
    rows[0].append('VARIANCE')
    for row in rows[1:]:
        vals = [float(x) for x in row[1:]]
        row.append(var(vals))
        
    print(tabulate(rows[1:], headers=rows[0]))
    
show_benchmarks('./code/benchmark/scripts/result.csv')

Name             bias_word2vec    debiased_word2vec    bias_glove    debiased_glove     VARIANCE
-------------  ---------------  -------------------  ------------  ----------------  -----------
AP                    0.557214             0.557214      0.532338          0.542289  0.000148511
BLESS                 0.67                 0.68          0.755             0.76      0.00228958
Battig                0.235519             0.233034      0.272032          0.267826  0.000427686
ESSLI_1a              0.727273             0.727273      0.772727          0.727273  0.000516529
ESSLI_2b              0.8                  0.8           0.75              0.75      0.000833333
ESSLI_2c              0.644444             0.644444      0.622222          0.622222  0.000164609
MEN                   0.704226             0.703828      0.76402           0.763982  0.00119901
MTurk                 0.513563             0.51474       0.640383          0.634242  0.00506272
RG65                  0.693775   

## Generating analogies

Once we have tested that the main properties of the embeddings are still in the debiased ones, then we can generate `he:she = x:y` analogies, and observe the resulting `x:y`  pairs.

### Analogies for original word2vec

In [29]:
!cd ./code/analogies/ && python3 analogies.py --pairs_fname pairs_bias_word2vec.txt --i_em ../../embeddings/bias_word2vec.bin --pair_seed he-she --n 100

/home/daddabarba/miniconda3/envs/fact/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
analogies.py:99: RuntimeWarning: invalid value encountered in true_divide
  analogies_dir = analogies/norms[:, None] # Directions
he is to she like pardon is to pardons
he is to she like grounds is to linking
he is to she like violin is to soprano
he is to she like peanut is to spinach
he is to she like storyteller is to educator
he is to she like indication is to indicates
he is to she like fastball is to hitter
he is to she like workspace is to workstation
he is to she like health_care is to care
he is to she like revival is to rebirth
he is to she like commercialization is to commercialized
he is to she like 

### Analogies for Debiased word2vec

In [30]:
!cd ./code/analogies/ && python3 analogies.py --pairs_fname pairs_debiased_word2vec.txt --i_em ../../embeddings/debiased_word2vec.bin --pair_seed he-she --n 100

/home/daddabarba/miniconda3/envs/fact/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
analogies.py:99: RuntimeWarning: invalid value encountered in true_divide
  analogies_dir = analogies/norms[:, None] # Directions
he is to she like subscriber is to subscription
he is to she like backfield is to rusher
he is to she like compound is to compounds
he is to she like sidestep is to avoid
he is to she like grade is to grades
he is to she like swipe is to swiping
he is to she like motherhood is to mothers
he is to she like sexual_harassment is to hazing
he is to she like sentence is to maximum_sentence
he is to she like grocer is to grocery_stores
he is to she like blues is to reggae
he is to she like 

### Analogies original glove

In [31]:
!cd ./code/analogies/ && python3 analogies.py --pairs_fname pairs_bias_glove.txt --i_em ../../embeddings/bias_glove.bin --pair_seed he-she --n 100

/home/daddabarba/miniconda3/envs/fact/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
analogies.py:99: RuntimeWarning: invalid value encountered in true_divide
  analogies_dir = analogies/norms[:, None] # Directions
he is to she like tuning is to tuned
he is to she like raise is to donate
he is to she like lunchtime is to brunch
he is to she like analogy is to cliche
he is to she like joke is to bitch
he is to she like chapel is to convent
he is to she like sustainability is to empowerment
he is to she like bunny is to girl
he is to she like emphasizing is to curricula
he is to she like battalion is to patrol
he is to she like dermatologist is to skincare
he is to she like snorkeling is to swim
h

### Analogies debiased glove

In [32]:
!cd ./code/analogies/ && python3 analogies.py --pairs_fname pairs_debiased_glove.txt --i_em ../../embeddings/debiased_glove.bin --pair_seed he-she --n 100

/home/daddabarba/miniconda3/envs/fact/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
analogies.py:99: RuntimeWarning: invalid value encountered in true_divide
  analogies_dir = analogies/norms[:, None] # Directions
he is to she like decal is to bumper
he is to she like peninsula is to harbour
he is to she like lotus is to lily
he is to she like refining is to perfecting
he is to she like sax is to soprano
he is to she like settling is to settles
he is to she like panel is to panels
he is to she like novel is to authors
he is to she like hoof is to horse
he is to she like officer is to magistrate
he is to she like demolition is to rubble
he is to she like midterm is to reelection
he is to she lik